##### 정형 데이터와 비정형 데이터
    - 정형 데이터 : 구조로 되어있는 데이터, csv나 엑셀, DB에 저장하기 쉬움
    - 비정형 데이터 : db나 엑셀로 표현하기 어려운 것들, 우리 주위의 카메라로 찍은 사진, 핸드폰으로 듣는 디지털 음악 등

In [1]:
#### 앙상블 학습 = 정형데이터를 다루는데 가장 성능이 좋음, 랜덤포레스트가 대표적

#### 랜덤 포레스트 : 앙상블 학습의 대표 주자 중 하나
                    - 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만든 다음 각 결정 트리의 예측을 사용해 최종 예측을 만든다
                    - 각 트리를 훈련하기 위한 데이터를 랜덤하게 만듦 (부트스트랩 샘플)
                    - 부트스트랩 샘플 : 1000개의 샘플이 들어있는 가방에서 100개의 샘플을 뽑는다면 먼저 1개를 뽑고,
                                        뽑았던 1개를 다시 가방에 넣음 이런식으로 계속 뽑으면 중복된 샘플을 뽑을 수 있음
                                        기본적으로 훈련세트의 크기와 같게 만든다
                    

In [2]:
import pandas as pd

df = pd.read_csv('https://bit.ly/wine_csv_data')

data = df[['alcohol', 'sugar', 'pH']]
target = df[['class']]

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = \
train_test_split(data, target)

##### RandomForestClassifier 
 전체 특성 개수의 제곱근만큼의 특성을 선택 (4개의 특성이 있다면 노드마다 2개를 랜덤하게 선택하여 사용) 
  - 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼는다
##### RandomForestRegressor = 전체 특성을 사용, 각 트리의 예측을 평균

##### RandomForestClassifier 
    - n_estimators : 앙상블을 구성할 트리의 개수를 지정, 기본값은 100
    - max_depth : 트리가 성장할 최대 깊이를 지정
    - min_samples_split : 노드를 나누기 위한 최소 샘플 개수, 기본값은 2 

In [4]:
from sklearn.model_selection import cross_validate # 교차 검증을 수행
from sklearn.ensemble import RandomForestClassifier # 기본적으로 100개의 결정 트리를 사용

rf = RandomForestClassifier(n_jobs=-1) 
scores = cross_validate(
    rf, train_input, train_target,
    return_train_score=True, n_jobs =-1 # return_train_score : 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환
)
print(scores)
# 훈련세트에 과대적합

{'fit_time': array([0.15042853, 0.25396943, 0.15629959, 0.14746094, 0.15559959]), 'score_time': array([0.03900528, 0.01573801, 0.02992654, 0.02926326, 0.02865911]), 'test_score': array([0.88102564, 0.88410256, 0.86550308, 0.89527721, 0.88398357]), 'train_score': array([0.99717732, 0.99871696, 0.99769112, 0.99743458, 0.99717804])}


In [5]:
rf.fit(train_input, train_target)
rf.feature_importances_
# alcohol = 0.23의 중요도, sugar = 0.49의 중요도, pH = 0.27의 중요도 
# 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문 

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


array([0.23044894, 0.49279823, 0.27675284])

#### OOB 샘플 : 부트스트랩 샘플에 포함되지 않고 남는 샘플
    - 이 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리 평가 가능, 검증 세트의 역할
    - oob_score = True로 지정해서 얻기

In [6]:
rf = RandomForestClassifier(oob_score = True, n_jobs=1) 
rf.fit(train_input, train_target)
rf.oob_score_

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.8920361247947455

#### ExtraTreesClassifier 
    - 랜덤 포레스트와 매우 비슷하게 동작
    - 기본적으로 100개의 결정 트리를 훈련
    - 부트스트랩 샘플을 사용하지 않음 
    - 결정 트리를 만들 때 전체 훈련 세트를 사용
    - 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할

In [7]:
from sklearn.ensemble import ExtraTreesClassifier
er = ExtraTreesClassifier()

scores = cross_validate(er, train_input, train_target, return_train_score = True)

print(scores)

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

{'fit_time': array([0.17251039, 0.19251752, 0.20312881, 0.19996881, 0.19782972]), 'score_time': array([0.01986337, 0.01992369, 0.02100539, 0.02108383, 0.0221715 ]), 'test_score': array([0.88923077, 0.88512821, 0.86550308, 0.88603696, 0.88501027]), 'train_score': array([0.99717732, 0.99871696, 0.99769112, 0.99769112, 0.99743458])}


#### GradientBoostingClassifier
    - 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
    - 기본적으로 깊이가 3인 결정 트리를 100개 사용
    - 경사 하강법을 사용하여 트리를 앙상블에 추가
    - 분류에서는 로지스틱 손실 함수를 사용, 회귀에서는 평균 제곱 오차 함수를 사용
    - 결정 트리를 계속 추가하면서 가장 낮은 곳을 찾아 이동
    - 랜덤 포레스트보다 조금 더 높은 성능을 얻지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느림

In [8]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators = 500) # n_estimaros = 나무를 500그루 심기
scores = cross_validate(
    gb, train_input, train_target,
    return_train_score = True
)
print(scores)
# 과대적합이 되지않음

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array w

{'fit_time': array([1.16945338, 1.17166662, 1.1711216 , 1.17437959, 1.26271605]), 'score_time': array([0.01049948, 0.00853062, 0.010988  , 0.00731707, 0.00752497]), 'test_score': array([0.8625641 , 0.87487179, 0.85728953, 0.88193018, 0.88706366]), 'train_score': array([0.92250449, 0.92301771, 0.93099025, 0.91995895, 0.91944587])}


#### HistGradientBoostingClassifier 
    - 입력 특성을 256개의 구간으로 나눠서 노드를 분할할m 때 최적의 분할을 매우 빠르게 찾을 수 있음
    - 그레이디언트 부스팅의 속도를 개선

In [11]:
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier() # max_iter 로 부스팅 반복 횟수 지정
scores = cross_validate(
    hgb, train_input, train_target,
    return_train_score =True
)
print(scores)

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\yerin\AppData\Local\Programs\Python\Python313\

{'fit_time': array([0.38345194, 0.34235525, 0.34039235, 0.33846092, 0.33990932]), 'score_time': array([0.00674987, 0.00656939, 0.0064919 , 0.00643468, 0.0067513 ]), 'test_score': array([0.86564103, 0.87794872, 0.86036961, 0.88911704, 0.89117043]), 'train_score': array([0.93199897, 0.9363613 , 0.94099538, 0.93329913, 0.93073371])}


In [12]:
# permutation_importance : 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산

from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
scores = permutation_importance(hgb, train_input, train_target) 
# 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 관찰
print(scores) 
# importances : 반복하여 얻은 특성 중요도
# 'importances_mean' : 평균 (가장 중요)

C:\Users\yerin\Desktop\DAMF\ML\venv\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'importances_mean': array([0.0952381 , 0.24035304, 0.09166667]), 'importances_std': array([0.00186319, 0.00626884, 0.00379715]), 'importances': array([[0.09462233, 0.0979064 , 0.09256979, 0.09667488, 0.09441708],
       [0.23522167, 0.24835796, 0.24671593, 0.23234811, 0.23912151],
       [0.09482759, 0.08723317, 0.09729064, 0.08866995, 0.09031199]])}


In [13]:
hgb.score(test_input, test_target)

0.8744615384615385